In [1]:
import pandas as pd
import datetime
import numpy as np
from pandas_datareader import data as web
from matplotlib import pyplot as pp

In [2]:
zscore = lambda x:(x -x.mean())/x.std() # zscore: normalization of log returns

In [3]:
start = datetime.datetime(2000,1,1)
end = datetime.datetime(2016,1,1)

In [4]:
df = web.DataReader("GOOGL", "google", start, end)

In [5]:
def _load_data(data, n_prev):  
    """
    data should be pd.DataFrame()
    """

    docX, docY = [], []
    for i in range(len(data)-n_prev):
        zscore(data.iloc[i:i+n_prev+1])
        docX.append(zscore(data.iloc[i:i+n_prev]).as_matrix())
        docY.append(zscore(data.iloc[i:i+n_prev+1]).Close[30])
    alsX = np.array(docX)
    alsY = np.array(docY)
    
    return alsX, alsY



In [6]:
def train_test_split(df, test_size=0.1):  
    """
    This just splits data to training and testing parts
    """
    ntrn = int(round(len(df) * (1 - test_size)))

    X_train, y_train = _load_data(df.iloc[0:ntrn],30)
    X_test, y_test = _load_data(df.iloc[ntrn:],30)

    return (X_train, y_train), (X_test, y_test)

In [7]:
(X_train, y_train), (X_test, y_test) = train_test_split(df)

In [15]:
from keras.models import Sequential  
from keras.layers.core import Dense, Activation, Dropout 
from keras.layers.recurrent import LSTM


hidden_neurons = 300

model = Sequential()
model.add(LSTM(300, input_shape=(30, 5), return_sequences=False))       
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation("tanh"))

model.compile(loss="mean_squared_error", optimizer="Adam",  metrics=['accuracy'])  

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 300)               367200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 301       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 367,501
Trainable params: 367,501
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(X_train, y_train, batch_size=100, epochs=40, validation_split=0.05)  

Train on 2418 samples, validate on 128 samples
Epoch 1/40
2418/2418 [==============================] - 9s - loss: 0.8426 - acc: 0.0000e+00 - val_loss: 0.6902 - val_acc: 0.0000e+00
Epoch 2/40
2418/2418 [==============================] - 8s - loss: 0.6457 - acc: 0.0000e+00 - val_loss: 0.6245 - val_acc: 0.0000e+00
Epoch 3/40
2418/2418 [==============================] - 8s - loss: 0.6119 - acc: 0.0000e+00 - val_loss: 0.6046 - val_acc: 0.0000e+00
Epoch 4/40
2418/2418 [==============================] - 8s - loss: 0.5923 - acc: 0.0000e+00 - val_loss: 0.5918 - val_acc: 0.0000e+00
Epoch 5/40
2418/2418 [==============================] - 8s - loss: 0.5872 - acc: 0.0000e+00 - val_loss: 0.5793 - val_acc: 0.0000e+00
Epoch 6/40
2418/2418 [==============================] - 8s - loss: 0.5782 - acc: 0.0000e+00 - val_loss: 0.5888 - val_acc: 0.0000e+00
Epoch 7/40
2418/2418 [==============================] - 9s - loss: 0.5779 - acc: 0.0000e+00 - val_loss: 0.5715 - val_acc: 0.0000e+00
Epoch 8/40
2418/2418 [

In [17]:
predicted = model.predict(X_test) 

In [18]:
outcome = pd.DataFrame()
outcome['actual'] = y_test
outcome['predicted'] = predicted
outcome.to_csv("outcome.csv")

In [19]:
metrics = model.evaluate(X_test, y_test)

256/256 [==============================] - 0s     
